In [ ]:
""" Extract bottleneck features using neural network. """

import os
import math
import sys
import torch
import pickle as cPickle
from os.path import join as pjoin
import torch.nn as nn
from torch.autograd import Variable
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from cnn.factory import get_model
from cnn.config import cfg
from datasets.folder import SpecImageFolder
from PIL import Image
import numpy as np


In [ ]:
data_path = "/mnt/data1/webvision2.0"
batch_size = 100 
input_size= 224
arch="resnet50"
gpus ="0,1,2,3" 
data_folder ="both" 
save_freq = 5000
num_classes= 5607 #number of class we want
print_freq= 10000
load_path ='/home/betty/webvision_train/results/resnet50/5000classes_onemonth/'
num_workers = 4
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
images_so_far = 0
save_path='/home/betty/webvision_train/results/resnet50/training_features'
params_file="/home/betty/webvision_train/results/resnet50/5000classes_onemonth/model_best.tar"


In [ ]:
queries=[]
previous=-1
all=True #to activate if you want all the query put in one class
if (num_classes is None):
    all=True
with open(os.path.join(data_path, 'info', 'queries_synsets_map.txt')) as f:
        for line in f:
            # Image path
            if((line.split()[1] != previous) or all):
                queries.append(int(line.split()[0]))
                previous=line.split()[1]
                #print(line.split()[0], " ", line.split()[1])
            elif ((len(queries)>=query) and not all):
                print("over the class", query, 'queries is', queries)
                break
query_synset_dic = {}
with open("/home/betty/Pictures/webvision/info/queries_synsets_map.txt") as f:
    for line in f:
       (key, val) = line.split()
       if (int(val)<=int(num_classes)):
           query_synset_dic[int(key)] = val
#print("We use ", query_synset_dic, "queries as synset map")
# print("class weighted", args.class_weighted )


In [ ]:
def save_to_pickle(features, save_path, cls_id, fname):

    with open(pjoin(save_path, cls_id, fname + ".pkl"), 'wb') as f:
        cPickle.dump(features, f, protocol=cPickle.HIGHEST_PROTOCOL)


def save_to_txt(img_tuple, save_path, cls_id):
    with open(pjoin(save_path, cls_id, "img_name.lst"), 'w') as f:
        for img_path, _ in img_tuple:
            f.write(img_path + '\n')

In [ ]:
def extract_feats(ext_loader, folder, model,
                  save_path, cls_id,
                  save_freq=200):
    # switch to evaluate mode
    save_path = pjoin(save_path, folder)
    model.eval()
    print(pjoin(save_path, cls_id))
    if not os.path.exists(pjoin(save_path, cls_id)): #save the resulting feature
        os.mkdir(pjoin(save_path, cls_id))

    batch_feat = []
    img_names = ext_loader.dataset.imgs
    batch_size = ext_loader.batch_size
    num_img = len(img_names)
    init_idx = 0
    pkl_idx = 0
    last_idx = int(math.ceil(num_img / float(batch_size))) - 1

    for idx, data in enumerate(ext_loader, 0):
        inputs, _ = data
        inputs = Variable(inputs.to(device))
        feats = model(inputs)
        print("check shape", feats.shape)

        cpu_feat = feats.data.cpu().numpy()
        if len(cpu_feat.shape) == 1:
            cpu_feat = np.reshape(cpu_feat, (1, -1))
        batch_feat.append(cpu_feat)

        if idx % save_freq == (save_freq - 1):
            # batch_im_list = img_names[
            #     init_idx: batch_size * save_freq + init_idx]
            # init_idx = batch_size * save_freq + init_idx
            batch_feat = np.concatenate(batch_feat, axis=0)
            save_to_pickle(batch_feat, save_path, cls_id, str(pkl_idx))

            batch_feat = []
            pkl_idx += 1

        elif idx == last_idx:
            # batch_im_list = img_names[init_idx:]
            batch_feat = np.concatenate(batch_feat, axis=0)

            save_to_pickle(batch_feat, save_path, cls_id, str(pkl_idx))

    # save to text
    save_to_txt(img_names, save_path, cls_id)

In [ ]:
def extract_model(data_root, gpus=gpus, batch_size=16,
                  params_file="/home/betty/webvision_train/results/resnet50/5000classes_onemonth/model_best.tar",
                  num_workers=4, num_classes=5000,
                  in_size=224, save_path=None, dict_=None):
    
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])

    ext_transform = transforms.Compose([
        transforms.Resize((in_size, in_size)),
        transforms.ToTensor(),
        normalize])
    print(params_file)
    assert os.path.isfile(params_file), "{} is not exist.".format(params_file)
    # define the model
    model = get_model(name=arch,
                      num_classes=num_classes, extract_feat=True)
    if len(gpus) > 1:
        prev_gpus = gpus
        gpus = [int(i) for i in gpus.strip('[]').split(',')]
        print("Let's use", gpus, "on", torch.cuda.device_count(), "GPUs!")
        os.environ["CUDA_VISIBLE_DEVICES"] =  prev_gpus #we give this number as the device id we use
        gpus_idx = range(0,len(gpus))
        model = torch.nn.DataParallel(model, device_ids=gpus_idx)
    elif len(gpus) == 1:
        prev_gpus = gpus
        gpus = [int(i) for i in gpus.strip('[]').split(',')]
        print("Let's use", gpus, "on", torch.cuda.device_count(), "GPUs!")
        os.environ["CUDA_VISIBLE_DEVICES"] =  prev_gpus #we give this number as the device id we use

    else:
        os.environ["CUDA_VISIBLE_DEVICES"] = gpus
        print("no gpus")

    model.to(device)
    model.cuda()
    params = torch.load(params_file)

    model.load_state_dict(params['state_dict'])
    #modules = list(model.children())[:-1]
    #model =  nn.Sequential(*modules)
    
    ##un comment to PRINT the model
#     print("Params to learn:")
#     feature_extract=False
#     if feature_extract:
#         params_to_update = []
#         for name,param in model.named_parameters():
#             if param.requires_grad == True:
#                 params_to_update.append(param)
#                 print("\t",name)
#     else:
#         for name,param in model.named_parameters():
#             if param.requires_grad == True:
#                 print("\t",name)
    feature_map = list(model.module.children())
    feature_map.pop()
    extractor = nn.Sequential(*feature_map)
    extractor.to(device)
    extractor.cuda()
    #print("feature_map", feature_map)
    for name,param in extractor.named_parameters():
            if param.requires_grad == True:
                print("\t",name)
    
#for every query
    for cls_id in sorted(os.listdir(data_root)): #args.data_path  
        print("Processing class {}".format(cls_id))
        ext_data = SpecImageFolder(root=pjoin(data_root, cls_id),
                                   transform=ext_transform, dict_=dict_)
        ext_loader = DataLoader(dataset=ext_data, batch_size=batch_size,
                                shuffle=False, num_workers=num_workers,
                                pin_memory=True)
        extract_feats(ext_loader, data_root[24:30], extractor, save_path,  cls_id)

In [ ]:

        
google_path="/mnt/data1/webvision2.0/google_images_resized"
flickr_path="/mnt/data1/webvision2.0/flickr_images_resized"
extract_model(google_path,
                  gpus,
                  batch_size,
                  params_file,
                  num_workers,
                  num_classes,
                  input_size,
                  save_path,dict_=query_synset_dic)